# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.08511


extracting tarball to tmp_2304.08511...

 done.


Found 68 bibliographic references in tmp_2304.08511/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2304.08613


extracting tarball to tmp_2304.08613...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.08613/ms_fred.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Found 120 bibliographic references in tmp_2304.08613/ms_fred.bbl.
syntax error in line 88: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.08511-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.08511) | **JWST/NIRSpec Measurements of Extremely Low Metallicities in High  Equivalent Width Lyman-$α$ Emitters**  |
|| M. V. Maseda, et al. -- incl., <mark>L. Boogaard</mark> |
|*Appeared on*| *2023-04-19*|
|*Comments*| *13 pages, 4 appendices; submitted to AAS Journals*|
|**Abstract**| Deep VLT/MUSE optical integral field spectroscopy has recently revealed an abundant population of ultra-faint galaxies ($M_{UV} = -$15; 0.01 $L_{\star}$) at $z=$2.9$-$6.7 due to their strong Lyman-$\alpha$ emission. The implied Lyman-$\alpha$ equivalent widths are in excess of 100-200 Angstrom, challenging existing models of normal star formation and implying extremely young ages, small stellar masses, and a very low amount of metal enrichment. We use JWST/NIRSpec's microshutter array to follow-up 45 of these galaxies (11h in G235M/F170LP and 7h in G395M/F290LP), as well as 45 lower-equivalent width Lyman-$\alpha$ emitters. Our spectroscopy covers the range 1.7$-$5.1 micron in order to target strong optical emission lines: H$\alpha$, [OIII], H$\beta$, and [NII]. Individual measurements as well as stacks reveal line ratios consistent with a metal poor nature (2$-$30% $Z_{\odot}$) and intense ionizing radiation fields. The galaxies with the highest equivalent widths of Lyman-$\alpha$, in excess of 120 Angstrom, have lower gas-phase metallicities than those with lower equivalent widths. This implies a selection based on Lyman-$\alpha$ equivalent width is an efficient technique for identifying younger, less chemically enriched systems. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.08613-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.08613) | **The Panchromatic Hubble Andromeda Treasury XX: The Disk of M31 is Thick**  |
|| J. J. Dalcanton, et al. -- incl., <mark>M. Fouesneau</mark> |
|*Appeared on*| *2023-04-19*|
|*Comments*| *22 pages. Accepted to the Astrophysical Journal*|
|**Abstract**| We present a new approach to measuring the thickness of a partially face-on stellar disk, using dust geometry. In a moderately-inclined disk galaxy, the fraction of reddened stars is expected to be 50% everywhere, assuming that dust lies in a thin midplane. In a thickened disk, however, a wide range of radii project onto the line of sight. Assuming stellar density declines with radius, this geometrical projection leads to differences in the numbers of stars on the near and far sides of the thin dust layer. The fraction of reddened stars will thus differ from the 50% prediction, with a deviation that becomes larger for puffier disks. We map the fraction of reddened red giant branch (RGB) stars across M31, which shows prominent dust lanes on only one side of the major axis. The fraction of reddened stars varies systematically from 20% to 80%, which requires that these stars have an exponential scale height h_z that is 0.14+/-0.015 times the exponential scale length (h_r~5.5kpc). M31's RGB stars must therefore have h_z=770+/-80pc, which is far thicker than the Milky Way's thin disk, but comparable to its thick disk. The lack of a significant thin disk in M31 is unexpected, but consistent with its interaction history and high disk velocity dispersion. We suggest that asymmetric reddening be used as a generic criteria for identifying ``thick disk'' dominated systems, and discuss prospects for future 3-dimensional tomographic mapping of the gas and stars in M31. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.08511.md
    + _build/html/tmp_2304.08511/./line_ratios_four.png
    + _build/html/tmp_2304.08511/./noise_comparison.png
exported in  _build/html/2304.08613.md
    + _build/html/tmp_2304.08613/./f_fredgrid.png
    + _build/html/tmp_2304.08613/./f_good_fred_map.png
    + _build/html/tmp_2304.08613/./f_bovy_hz.png
    + _build/html/tmp_2304.08613/./f_bovy_hzhr.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\minus}{\scalebox{0.5}[1.0]{-}}$
$\newcommand{\kms}{\>{\rm km}~ {\rm s}^{-1}}$
$\newcommand{\reff}{r_{\rm{eff}}}$
$\newcommand{\msol}{M_{\odot}}$
$\newcommand{\zsol}{Z_{\odot}}$
$\newcommand{\inverse}[1]{{#1}^{-1}}$
$\newcommand{\invvar}{\inverse{C}}$
$\newcommand{\dd}{{\rm d}}$
$\newcommand{\cgs}{erg s^{-1} cm^{-2}}$
$\newcommand{\pf}{\texttt{pyPlatefit}}$
$\newcommand{\magphys}{\texttt{MAGPHYS} }$
$\newcommand{\lya}{Lyman-\alpha }$
$\newcommand{\lyans}{Lyman-\alpha}$
$\newcommand{\oi}{[O I]}$
$\newcommand{\sii}{[S II]}$
$\newcommand{\nii}{[\ion{N}{2}]}$
$\newcommand{\neiii}{[\ion{Ne}{3}]}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\oiii}{[\ion{O}{3}]}$
$\newcommand{\oii}{[\ion{O}{2}]}$
$\newcommand{\ewoii}{EW_{\mathrm{[O II]}}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\df}{D_{n}4000}$
$\newcommand{\hd}{H\delta}$
$\newcommand{\hda}{H\delta_{A}}$</div>



<div id="title">

# JWST/NIRSpec Measurements of Extremely Low Metallicities in High Equivalent Width Lyman-$\alpha$ Emitters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.08511-b31b1b.svg)](https://arxiv.org/abs/2304.08511)<mark>Appeared on: 2023-04-19</mark> -  _13 pages, 4 appendices; submitted to AAS Journals_

</div>
<div id="authors">

M. V. Maseda, et al. -- incl., <mark>L. Boogaard</mark>

</div>
<div id="abstract">

**Abstract:** Deep VLT/MUSE optical integral field spectroscopy has recently revealed an abundant population of ultra-faint galaxies ( $M_{UV} \approx -15$ ; 0.01 $L_{\star}$ ) at $z=$ 2.9 $-$ 6.7 due to their strong Lyman- $\alpha$ emission with no detectable continuum.  The implied $\lya$ equivalent widths can be in excess of 100-200 Å, challenging existing models of normal star formation and indicating extremely young ages, small stellar masses, and a very low amount of metal enrichment. We use JWST/NIRSpec to follow-up 45 of these galaxies (11h in G235M/F170LP and 7h in G395M/F290LP), as well as 45 lower-equivalent width Lyman- $\alpha$ emitters.  Our spectroscopy covers the range 1.7 $-$ 5.1 micron in order to target strong optical emission lines: $\ha$ , $\oiii$ , $\hb$ , and $\nii$ . Individual measurements as well as stacks reveal line ratios consistent with a metal poor nature (2 $-$ 30 \% $Z_{\odot}$ , depending on the calibration).  The galaxies with the highest equivalent widths of $\lyans$ , in excess of 120 Å, have lower gas-phase metallicities than those with lower equivalent widths.  This implies a selection based on $\lyans$ equivalent width is an efficient technique for identifying younger, less chemically enriched systems.

</div>

<div id="div_fig1">

<img src="tmp_2304.08511/./line_ratios_four.png" alt="Fig3" width="100%"/>

**Figure 3. -** (Upper left) R3 versus N2 ionization diagnostic plot for our sample of LAEs at $z\approx4.6$.  For objects with S/N $<$ 1 in any of the individual lines, we use gray triangles to represent limits (2-$\sigma$) on the ratio(s); objects without a S/N $>$ 1 detection in both components of R3 or N2 are omitted, although they are still included in the stacks.  The stacked values from our LAEs (large circles) lie below the sequence of $z\approx2.3$ star-forming galaxies (SFGs) from [Strom, Steidel and Rudie (2017)](), the [Sanders, et. al (2023)](), including the $\pm$1-$\sigma$ spread (thin lines).  The shaded regions show the stacked R3 ranges, including those from [Sanders, et. al (2023)](). Our LAEs have lower R3 values than the "typical" SFGs from [Sanders, et. al (2023)](), indicating lower metallicities (vertical dashed lines). (*fig:lya-r3*)

</div>
<div id="div_fig3">

<img src="tmp_2304.08511/./noise_comparison.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of the background-subtracted signal-to-noise values for (left) all pixels in the level 1 processed "rate" files, (center) pixels illuminated by open MSA shutters in the level 2 "cal" files, and (right) pixels in the \texttt{Pypeit}-calibrated 2D frames, including the extra factors of 1.85 and 1.62 in the noise estimates as described in the text.  Each of the two detectors are plotted separately.  The discrepancy in the distributions of the levels 1 and 2 products compared to Gaussian statistics (dashed line) motivate the extra noise factors used in \texttt{Pypeit}. (*fig:noise*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.08511"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# The Panchromatic Hubble Andromeda Treasury XX: The Disk of M31 is Thick

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.08613-b31b1b.svg)](https://arxiv.org/abs/2304.08613)<mark>Appeared on: 2023-04-19</mark> -  _22 pages. Accepted to the Astrophysical Journal_

</div>
<div id="authors">

J. J. Dalcanton, et al. -- incl., <mark>M. Fouesneau</mark>

</div>
<div id="abstract">

**Abstract:** We present a new approach to measuring the thickness of a partially face-on stellar disk, using dust geometry. In a moderately-inclined disk galaxy, the fraction of reddened stars is expected to be 50 \% everywhere, assuming that dust lies in a thin midplane.  In a thickened disk, however, a wide range of radii project onto the line of sight. Assuming stellar density declines with radius, this geometrical projection leads to differences in the numbers of stars on the near and far sides of the thin dust layer. The fraction of reddened stars will thus differ from the 50 \% prediction, with a deviation that becomes larger for puffier disks.  We map the fraction of reddened red giant branch (RGB) stars across M31, which shows prominent dust lanes on only one side of the major axis. The fraction of reddened stars varies systematically from 20 \% to 80 \% , which requires that these stars have an exponential scale height $h_z$ that is $0.14\pm0.015$ times the exponential scale length ( $h_r\approx5.5\kpc$ ). M31's RGB stars must therefore have $h_z=770\pm80\pc$ , which is far thicker than the Milky Way's thin disk, but comparable to its thick disk. The lack of a significant thin disk in M31 is unexpected, but consistent with its interaction history and high disk velocity dispersion. We suggest that asymmetric reddening be used as a generic criteria for identifying "thick disk" dominated systems, and discuss prospects for future 3-dimensional tomographic mapping of the gas and stars in M31.

</div>

<div id="div_fig1">

<img src="tmp_2304.08613/./f_fredgrid.png" alt="Fig7" width="100%"/>

**Figure 7. -** Maps of the apparent fraction of reddened stars for models of
  inclined, thickened disks where the dust is assumed to be confined
  to the midplane with negligible thickness compared to the stars
  (i.e., $h_z >> h_{dust}$). The apparent inclinations of the model
  disks increases from left to right ($i=15◦ee$, $30◦ee$,
  $45◦ee$, $60◦ee$, \&$75◦ee$), and the disk thickness
  increases from bottom to top ($h_z/h_r=0.05$, 0.1, 0.15, \& 0.2). As
  expected, the fraction of reddened stars is always 50\% along the
  major axis, but deviates strongly perpendicularly, with the largest
  deviations seen for higher inclinations and intrinsically thicker
  disks.  Contours indicate deviations of $\pm5$\%, $\pm10$\%,
  $\pm20$\%, \&$\pm30$\% relative to 50\%, with thicker contours
  indicating larger deviations.
 (*fredgridfig*)

</div>
<div id="div_fig2">

<img src="tmp_2304.08613/./f_good_fred_map.png" alt="Fig10" width="100%"/>

**Figure 10. -** Map of the fraction of reddened stars $f_{red}$, restricted
  to high extinction ($A_V>1.25\mags$) regions with well-measured
  values of $f_{red}$($\Delta f_{red} < 0.06$). There is a clear
  gradient in the fraction of reddened stars from the far side to the
  near side of the disk (i.e., left to right).  The majority of the
  PHAT survey area, which was initially targeted to avoid M31's dust
  lanes, has fewer than $\sim$25\% of its old stellar population
  behind the dusty ISM.  The center of M31 is in the lower right, at RA$\approx$10.68 and Dec$\approx$41.27.
 (*fredmapfig*)

</div>
<div id="div_fig3">

<img src="tmp_2304.08613/./f_bovy_hz.png" alt="Fig11.1" width="50%"/><img src="tmp_2304.08613/./f_bovy_hzhr.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Comparison between the structure of M31's RGB disk and the
  structure of the Milky Way's monoabundance populations from
  \citet{bovy2012}.  (Left) Monoabundance populations in [Fe/H] vs
        [$\alpha$/Fe], color-coded by their vertical scale height at
        the solar circle.  Populations whose scale height falls in the
        range allowed by the data in M31 are outlined in black.  Only
        low metallicity, $\alpha$-enhanced Milky Way populations have
        scale heights comparable to those we see in M31.  (Right) The
        axial ratio of the monoabundance subpopulations as a function
        of [Fe/H] calculated as a mass-weighted average of $h_z/h_r$
        over [$\alpha$/Fe] at fixed [Fe/H]. Points are color-coded by
        the fraction of the integrated local stellar density found in
        populations with metallicities equal to or lower than [Fe/H].
        The horizontal bar show the likely range of $h_z/h_r$ for
        M31's stellar disk.  Only low metallicity populations
        ([Fe/H]$\lesssim-0.5$) that make up $\sim$30\% of the Milky
        Way disk are as puffy as the M31 disk.
 (*bovyfig*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.08613"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

192  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
